<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Assessment Warm-Up:</b> Creating A Basic Retriever Node</h2>
<br>

We've now been introduced to ReAct as a concept, and could make a toy system that exhibits this property. However, focusing on it as the ideal paradigm isn't necessarily always correct. It is extremely flexible and does have its purposes. When organized by an overarching strong LLM, this type of loop can run for quite a while since the tools can be used to hide details from the main loop. Integrate this system in with some context re-canonicalization step, and you could theorhetically go on forever.

From an implementation perspective, creating a coherent system with this is actually quite simple in principle. It's a good exercise, but isn't worth the effort to build in this course as it doesn't show off any new features:

> **HINT:** It's the agent loop from Section 3, but the LLM is bound to call functions, the stop condition is when no tool is called, and some effort is needed to make sure the tool responses actually help to enforce a valid prompting strategy.

This paradigm is great for ***horizontal agents*** and ***supervisor-style nodes***, where you can keep tossing more functions ("ways to interface with the environment") at the LLM while hoping it will pick something up. Hence, why this pattern works better with a stronger LLM where the state of "it just works" is easier to achieve.

In this notebook, we will try implementing a ***tool-like agent*** system which is tweaked for the specific problem and aims to hide its runtime details from any other main event loops which may be overseeing it. (i.e. the user, an overarching ReAct loop, some other supervisor, etc). In doing so, we will rediscover some of the interfaced from the RAG course while recontextualizing them into our LangGraph workflows.

**This exercise is specifically intended to prepare you for the assessment!**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA

from transformers import PreTrainedTokenizerFast
llama_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json", clean_up_tokenization_spaces=True)
def token_len(text):
    return len(llama_tokenizer.encode(text=text))

# !pip install --upgrade langgraph colorama
llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://llm_client:9000/v1")

<hr><br>

## **Part 1:** Pulling In Some Boilerplate

Let's start off by pulling in our old-reliable base specification for a simple multi-turn system. To make this and subsequent processes easier, we will switch to using an entirely Command-based routing scheme, and will try to reuse components as they need integration.

In [ ]:
import uuid
from typing import Annotated, Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.graph.message import add_messages
from functools import partial
from colorama import Fore, Style
from copy import deepcopy
import operator
from course_utils import stream_from_app

##################################################################

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
##################################################################

def user(state: State):
    update = {"messages": [("user", interrupt("[User]:"))]}
    return Command(update=update, goto="agent")
    
def agent(state: State, config=None):
    update = {"messages": [llm.invoke(state.get("messages"), config=config)]}
    if "stop" in state.get("messages")[-1].content: 
        return update
    return Command(update=update, goto="start")
    
##################################################################

builder = StateGraph(State)
builder.add_node("start", lambda state: {})
builder.add_node("user", user)
builder.add_node("agent", agent)
builder.add_edge(START, "start")
builder.add_edge("start", "user")
app = builder.compile(checkpointer=MemorySaver())
config = {"configurable": {"thread_id": uuid.uuid4()}}
app_stream = partial(app.stream, config=config)

for token in stream_from_app(app_stream, verbose=False, debug=False):
    print(token, end="", flush=True)

<br>

In this notebook, we will be combining our simple LangGraph app with the logic of our DLI Instructor prompt from earlier. You may recall that implementation looked something like the following:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA
from langchain_openai import ChatOpenAI
from functools import partial

## Back-and-forth loop
core_prompt = ChatPromptTemplate.from_messages([
    ("system",
         "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
         " Please help to answer user questions about the course. The first message is your context."
         " Restart from there, and strongly rely on it as your knowledge base. Do not refer to your 'context' as 'context'."
    ),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("placeholder", "{messages}")
])

## Am LCEL chain to pass into chat_with_generator
chat_chain = core_prompt | llm | StrOutputParser()

with open("simple_long_context.txt", "r") as f:
    full_context = f.read()

long_context_state = {
    "messages": [],
    "context": full_context,
}

from course_utils import chat_with_chain

chat = partial(chat_with_chain, chain=chat_chain)
chat(long_context_state)

<br>

You may also recall that this system was only able to take a couple of questions at a time because the context length would quickly exceed the deployed model's limits. We will try to fix that as part of our exercise!

<hr><br>

## **Part 2:** Filtering Out The Details

Understanding that the context length of our content chatbot is too limited, you may feel included to refine it some more and get down to an even smaller context, but our current entries are already relatively short. 

In [ ]:
from langchain_core.documents import Document

context_entries = full_context.split("\n\n")
context_docs = [Document(page_content=entry) for entry in context_entries if len(entry.split("\n")) > 2]
context_lens = [token_len(d.page_content) for d in context_docs]
print(f"Context Token Length: {sum(context_lens)} ({sum(context_lens)/len(context_lens):.2f} * {len(context_lens)})")
print(f"Document Token Range: [{min(context_lens)}, {max(context_lens)}]")

Perhaps we could invoke some heuristic to help us know which ones to focus on for any given question. Lucky for us, we have several viable heuristics in the form of **embedding models**! These have been covered at length in other courses, so here's just a high-level overview:

**Instead of *autoregressing* a sequence out of another sequence as a response/continuation, an encoder *embeds* the sequence into a per-token embedding, of which a subset (zero-th entry, subset, entire sequence) is used as a semantic encoding of the input.** Let's see which model options we have at our displosal.

- A **reranking model** orders a set of document pairs by relevance as its default behavior. This style of model is usually implemented with a *cross-encoder*, which takes both sequences as input and directly predicts a relevance score while actively considering both sequences.
- An **embedding model** embeds a document into a semantic embedding space as its default behavior. This style of model is usually implemented with a *bi-encoder*, which takes in one sequence at a time to produce the embedding. However, two embedded entries can be compared using some similarity metric (i.e. cosine similarity).

Either model could technically be used for retrieval, so let's go ahead and try both options!

In [ ]:
from langchain_nvidia import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## First, we can try out the embedding model, which is commonly used by first constructing a vectorstore.
## - Pros: If you have m documents and n queries, you need n inference-time embeddings and m*n similarity comparisons. 
## - Cons: Prediction of d_i sim q_j uses learned embeddings Emb_D(d_i) and Emb_Q(q_i),
##         not a joint learned representation Emb(d_i, q_j). In other words, somewhat less accurate.

question = "Can you tell me about multi-turn agents?"

embed_d = NVIDIAEmbeddings(model="nvidia/llama-3.2-nv-embedqa-1b-v2", base_url='http://llm_client:9000/v1', truncate='END', max_batch_size=128)
embed_q = NVIDIAEmbeddings(model="nvidia/llama-3.2-nv-embedqa-1b-v2", base_url='http://llm_client:9000/v1', truncate='END', max_batch_size=128) ## Not necessary
vectorstore = FAISS.from_documents(context_docs, embed_d)
vectorstore.embedding_function = embed_q
retriever = vectorstore.as_retriever()
%time retriever.invoke(question, k=5)
# %time retriever.invoke(question, k=1)

In [ ]:
from langchain_nvidia import NVIDIARerank

## Next, we can try out the reranking model, which is queried directly to get predicted relevance scores.
## - Pros: Literally predicts Emb(d_i, q_i), so better joint relationships can be learned. 
## - Cons: If you have m documents and n queries, you need n*m inference-time embeddings. 

question = "Can you tell me about multi-turn agents?"

reranker = NVIDIARerank(model="nvidia/llama-3.2-nv-rerankqa-1b-v2", base_url='http://llm_client:9000/v1', top_n=5, max_batch_size=128)
%time reranker.compress_documents(context_docs, question)

In [ ]:
# reranker._client.last_inputs
# reranker._client.last_response.json()
# embed_d._client.last_inputs
# embed_d._client.last_response.json()
# embed_q._client.last_inputs
# embed_q._client.last_response.json()

<br>

As we can see, this process is very fast at identifying similarities and produces pretty good rankings for this small data pool! More generally:
- **The reranking model is greatly preferred when we're dealing with a small pool of values,** since it leverage joint conditioning.
- **The embedding model is greatly preferred when dealing with large document pools,** since we can offload much of the embedding burden to the preprocessing stage.

For our limited use-case, the choice won't really matter, and you are free to use whichever option you find most compelling. With that said, go ahead and define a `retrieve` function to abstract this decision away. Furthermore, to streamline our handling later down the road, let's also return just the final string content so that we have less problems to worry about later. 

In [ ]:
def retrieve_via_query(query: str, k=5):
    reranker = NVIDIARerank(model="nvidia/llama-3.2-nv-rerankqa-1b-v2", base_url='http://llm_client:9000/v1', top_n=k, max_batch_size=128)
    rets = reranker.compress_documents(context_docs, query)
    return [entry.page_content for entry in rets]

retrieve_via_query(question)

<br>

From here, we can either make it into a "schema function," a "tool," or a "node," with the following key destinctions:
- A **schema function** can be bound to an LLM to force the output to the schema unconditionally.
- A **tool** is also a schema function, but is defined implicitly (i.e. structure of input is implied from signature) and is easier to toss into a tolbank.
- A **node** operates on and writes to the state buffer of a graph, so it should take in a `state` + `config`, operate on state variables, and output a state buffer modification request.

In this exercise, we're actually going to use the retrieval as an always-on feature of the "retrieval" agent, so we can bypass the first two and directly make our node function. Let's assume:
- We want our node to perform a retrieval on the previous message (i.e. we want the retrieval to happen AFTER the user submits a message, and we want to retrieve based on whatever the user sent).
- We want to write the retrieval results to a value `context` to the state buffer, as we will want the next node (an LLM generation) to use the context.
    - And we will want to accumulate `context` over time to include all relevant retrievals. That way, we can put the retrieval into the system message and it can continue to contribute to all subsequent outputs. This implies we'll want to store the values in a set...

In [ ]:
def retrieval_node(state: State, config=None, out_key="context"):
    ## NOTE: Very Naive; Assumes user question is a good query
    ret = retrieve_via_query(state.get("messages")[-1].content, k=3)
    return {out_key: set(ret)}

## After we define the node, we can assess whether or not it would work.

## Given an initial empty state...
state = {
    "messages": [], 
    "context": set(),
}

## Given an update rule explaining how to handle state updates...
add_sets = (lambda x,y: x.union(y))

## Will the continued accumulation of messages, followed by a continued accumulation of retrievals, function properly?
state["messages"] = add_messages(state["messages"], [("user", "Can you tell me about agents?")])
state["context"] = add_sets(state["context"],  retrieval_node(state)["context"])
print(f"Retriever: {state['context']} ({len(state['context'])})")

state["messages"] = add_messages(state["messages"], [("user", "How about earth simulations?")])
state["context"] = add_sets(state["context"],  retrieval_node(state)["context"])
print(f"\nContext: {state['context']} ({len(state['context'])})")

state["messages"] = add_messages(state["messages"], [("user", "How about earthly agents?")])
state["context"] = add_sets(state["context"],  retrieval_node(state)["context"])
print(f"\nContext: {state['context']} ({len(state['context'])})")

<hr><br>

## **Part 3:** Adding Retrieval To Our Graph 

Now that we have a generally-applicable node with some base assumptions, let's integrate it into our dialog loop from earlier and see if it just works.

In [ ]:
import uuid
from typing import Annotated, Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.graph.message import add_messages
from functools import partial
from colorama import Fore, Style
from copy import deepcopy
import operator

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    messages: Annotated[list, add_messages]
    context: Annotated[set, (lambda x,y: x.union(y))]

agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
         "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
         " Please help to answer user questions about the course. The first message is your context."
         " Restart from there, and strongly rely on it as your knowledge base. Do not refer to your 'context' as 'context'."
    ),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("placeholder", "{messages}")
])
    
##################################################################

def user(state: State):
    update = {"messages": [("user", interrupt("[User]:"))]}
    return Command(update=update, goto="retrieval_router")

## TODO: Add the retrieval between user and agent
def retrieval_router(state: State):
    return Command(update=retrieval_node(state), goto="agent")
    
def agent(state: State, config=None):
    update = {"messages": [(agent_prompt | llm).invoke(state, config=config)]}
    if "stop" in state.get("messages")[-1].content: 
        return update
    return Command(update=update, goto="start")
    
##################################################################

builder = StateGraph(State)
builder.add_node("start", lambda state: {})
builder.add_node("user", user)
## TODO: Register the new router to the nodepool
builder.add_node("retrieval_router", retrieval_router)
builder.add_node("agent", agent)
builder.add_edge(START, "start")
builder.add_edge("start", "user")
app = builder.compile(checkpointer=MemorySaver())
config = {"configurable": {"thread_id": uuid.uuid4()}}
app_stream = partial(app.stream, config=config)

for token in stream_from_app(app_stream, verbose=False, debug=False):
    print(token, end="", flush=True)

<hr>

As you can see, it wasn't too hard to integrate with the way this graph system is defined. We now have an "always-on" retrieval system which is always going to naively take our last message and retrieve the most relevant resources for our query... allegedly. However, if you play around with it a bit, you should start to notice that the raw input may not be exactly optimal, so most setups like to first rephrase the input into the canonical input form for the embedding model... but that would introduce latency and increase the time-to-first-token, making our system less responsive. 



<br><hr>

## **Part 4:** Adding A "Think Deeper" Mechanism

In this section, we're going to take some minor inspiration from ReAct to give our system multiple levels of thoroughness. Since our retrieval above was so light and is probably sufficient for most use-cases, let's keep it in. However, let's add a more rigorous thinking process that forces both a **query refinement** and a **web search** as part of its execution. 

This type of mechanism is often called a "reflection" mechanism since it can evaluate the output of the LLM and try to correct the execution flow. It largely works from the intuition that it's easier to verify if an output looks good than to generate the output in the first place.

We can achieve the querying logic with a single structured output schema, which we can test out below:

In [ ]:
from course_utils import SCHEMA_HINT
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict

class Queries(BaseModel):
    """Queries to help you research across semantic and web resources for more information. Specifically focus on the most recent question."""
    big_questions: List[str] = Field(description="Outstanding questions that need research, in natural language")
    semantic_queries: List[str] = Field(description="Questions (3 or more) to ask an expert to get more info to help, expressed in different ways.")
    web_search_queries: List[str] = Field(description="Questions (3 or more) that will be sent over to a web-based search engine to gather info.")

def query_node(state: State):
    if not state.get("messages"): return {"queries": []}
    chat_msgs = [
        ("system", SCHEMA_HINT.format(schema_hint = Queries.model_json_schema())),
        ("user", "Corrent Conversation:\n" + "\n\n".join([f"[{msg.type}] {msg.content}" for msg in state.get("messages")])),
    ]
    schema_llm = llm.with_structured_output(schema=Queries.model_json_schema(), strict=True)
    response = Queries(**schema_llm.invoke(chat_msgs))
    return {"queries": [response]}

add_queries = (lambda l,x: l+x) 

state = {
    "messages": [], 
    "queries": [],
}
state["messages"] = add_messages(state["messages"], [("user", "Can you tell me about agents?")])
state["queries"] = add_queries(state["queries"],  query_node(state)["queries"])
print("Queries:", state["queries"])

state["messages"] = add_messages(state["messages"], [("user", "How about earth simulations?")])
state["queries"] = add_queries(state["queries"],  query_node(state)["queries"])
print("\nQueries:", state["queries"])

<br>

Now, we have to actually fulfill those requests, so let's bring in both our retrieval function from this notebook and our DDGS search tool from the previous notebook and actually fulfill those requests.

In [ ]:
## HINT: You can paste the retrieval node and search tools directly and just resolve them in fulfill_query

# from langchain.tools import tool
# 
# @tool
# def search_internet(user_question: List[str], context: List[str], final_query: str):
#     """Search the internet for answers. Powered by search engine, in Google search format."""
#     from ddgs import DDGS
#     return DDGS().text(final_query, max_results=10)

# def retrieval_node(state: State, config=None, out_key="context"):
#     ## NOTE: Very Naive; Assumes user question is a good query
#     ret = retrieve_via_query(get_nth_message(state, n=-1), k=3)
#     return {out_key: set(ret)}

def fulfill_queries(queries: Queries, verbose=False):
    # big_questions: List[str]
    # semantic_queries: List[str]
    # web_search_queries: List[str]
    from ddgs import DDGS
    web_queries = queries.web_search_queries + queries.big_questions
    sem_queries = queries.semantic_queries + queries.big_questions
    # if verbose: print(f"Querying for retrievals via {web_queries = } and {sem_queries = }")
    web_ret_fn = lambda q: [
        str(f"{v.get('body')} [Snippet found from '{v.get('title')}' ({v.get('href')})]") 
        for v in DDGS().text(q, max_results=4)
    ]
    sem_ret_fn = retrieve_via_query
    web_retrievals = [web_ret_fn(web_query) for web_query in web_queries]
    sem_retrievals = [sem_ret_fn(sem_query) for sem_query in sem_queries]
    # if verbose: print(f"Generated retrievals: {web_retrievals = } and {sem_retrievals = }")
    return set(sum(web_retrievals + sem_retrievals, []))

retrievals = set()
new_rets = fulfill_queries(state["queries"][0], verbose=True)
retrievals = retrievals.union(new_rets)
print(f"Retrieved {len(new_rets)} chunks from the internet and the knowledge base")
new_rets

<br>

And perfect! We now have an unusably-long context that is, admittedly, better-thought-out but intractably long. Lucky for us, we have a pretty streamlined way of subsetting this with our retriever system, if only we generalize it a bit more.

In the following cell, please implement a `format_retrieval` function to create the actual context for the system.

In [ ]:
def filter_retrieval(
    queries: Queries, 
    new_retrievals: list[str], 
    existing_retrievals: set[str] = set(), 
    k=5
):
    # big_questions: List[str]
    # semantic_queries: List[str]
    # web_search_queries: List[str]
    reranker = NVIDIARerank(model="nvidia/llama-3.2-nv-rerankqa-1b-v2", base_url='http://llm_client:9000/v1', top_n=(k + len(existing_retrievals)), max_batch_size=128)
    docs = [Document(page_content = ret) for ret in new_retrievals]
    rets = reranker.compress_documents(docs, "\n".join(queries.big_questions))
    return [entry.page_content for entry in rets if entry.page_content not in existing_retrievals][:k]

filtered_retrieval = filter_retrieval(state["queries"][0], new_rets)
filtered_retrieval

<br>

And to wrap all of that up, proceed to make a single unified node call that executes this process as part of the routine, preferably without ever writing to the state buffer until the final new retrievals are generated.

**We will leave the final combination as an exercise, but the solution is provided for those interested.** After all, this should be prep for the assessment.

In [ ]:
import uuid
from typing import Annotated, Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command, Send
from langgraph.graph.message import add_messages
from functools import partial
from colorama import Fore, Style
from copy import deepcopy
import operator

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    messages: Annotated[list, add_messages]
    context: Annotated[set, (lambda x,y: x.union(y))]

agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
         "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
         " Please help to answer user questions about the course. The first message is your context."
         " Restart from there, and strongly rely on it as your knowledge base. Do not refer to your 'context' as 'context'."
         " If you think nothing in the context accurately answers your question and you should search deeper,"
         " include the exact phrase 'let me search deeper' in your response to perform a web search."
    ),
    ("user", "<context>\n{context}</context>"),
    ("ai", (
        "Thank you. I will not restart the conversation and will abide by the context."
        " If I need to search more, I will say 'let me search deeper' near the end of the response."
    )),
    ("placeholder", "{messages}")
])
    
##################################################################

def user(state: State):
    update = {"messages": [("user", interrupt("[User]:"))]}
    return Command(update=update, goto="retrieval_router")

## TODO: Add the retrieval between user and agent
def retrieval_router(state: State):
    return Command(update={"context": ""}, goto="agent")

def agent(state: State, config=None):
    if "END" in state.get("messages")[-1].content: 
        return {"messages": []}
    update = {"messages": [(agent_prompt | llm).invoke(state, config=config)]}
    if "New Context Retrieved:" in state.get("messages")[-1].content:
        pass
    elif "let me search deeper" in update['messages'][-1].content.lower():
        return Command(update=update, goto="deep_thought_node")
    return Command(update=update, goto="start")

def deep_thought_node(state: State, config=None):
    ## NOTE: Very Naive; Assumes user question is a good query
    deeper_queries = query_node(state)['queries'][0]
    new_rets = fulfill_queries(deeper_queries, verbose=True)
    new_rets = filter_retrieval(deeper_queries, new_rets, state.get("context"))
    update = {"messages": [("user", f"New Context Retrieved: {new_rets}")]}
    return Command(update=update, goto="agent")
    
##################################################################

builder = StateGraph(State)
builder.add_node("start", lambda state: {})
builder.add_node("user", user)
## TODO: Register the new nodes to the nodepool
builder.add_node("retrieval_router", retrieval_router)
builder.add_node("deep_thought_node", deep_thought_node)
builder.add_node("agent", agent)
builder.add_edge(START, "start")
builder.add_edge("start", "user")
app = builder.compile(checkpointer=MemorySaver())
config = {"configurable": {"thread_id": uuid.uuid4()}}
app_stream = partial(app.stream, config=config)

for token in stream_from_app(app_stream, verbose=False, debug=False):
    print(token, end="", flush=True)

<details>
    <summary><b>HINT:</b></summary>
    <code>retrieval_router</code> is currently manually injecting a context of "", so maybe we can just run the call to our retrieval function with a minimal amount of wrapping?  
</details>

<details>
    <summary><b>SOLUTION:</b></summary>

```python
## TODO: Add the retrieval between user and agent
def retrieval_router(state: State):
    return Command(update=retrieval_node(state), goto="agent")

def retrieval_node(state: State, config=None, out_key="context"):
    ## NOTE: Very Naive; Assumes user question is a good query
    ret = retrieve_via_query(state.get("messages")[-1].content, k=3)
    return {out_key: set(ret)}
```

</details>

<hr><br>

### **Part 5:** Reflecting On This Exercise

And just like that, we have some semblance of a ReAct-style loop, if in a much more limited capacity. Though it's not a "pool of tools" approach, it is definitely a "reflection system" with build-in routing. It also isn't really a proper "deep researcher" since it doesn't actually read the full body of the articles and isn't capable of expanding on the material quite yet, but it does exhibit very basic retrieval simplification to enable a much longer exchange window.

**In the next section, get ready to try out the assessment where you will be implementing a reasoning and searching feature based on the techniques presented in this notebook!**

<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>